In [35]:
import ccxt.async_support as ccxt
from ccxt.async_support.binance import binance
from typing import cast
import os
from pydantic import BaseSettings

assert os.getcwd() == "/app/jupyter"

class APICredentialBinance(BaseSettings):
    API_BINANCE_API_KEY: str
    API_BINANCE_API_SECRET: str

    class Config:
        env_file = '../.env'
        env_file_encoding = 'utf-8'


credential = APICredentialBinance()




In [64]:
import asyncio

# https://github.com/ccxt/ccxt
exchange_id = "binance"
exchange_class = getattr(ccxt, exchange_id)

def create_api() -> binance:
    return exchange_class({
    'apiKey': credential.API_BINANCE_API_KEY,
    'secret': credential.API_BINANCE_API_SECRET,
    'timeout': 30000,
    'enableRateLimit': True,
    })

api: binance = create_api()


async def main():
    await api.load_markets()
    pairs = [x for x in api.symbols if "BTC" in x]
    symbol = "BTC/USDT"
    ticker = await api.fetch_ticker(symbol)  # get_ticker
    # https://github.com/ccxt/ccxt/issues/6345
    # https://github.com/ccxt/ccxt/wiki/Manual#api-method-naming-conventions  implicit rule
    # result = await api.sapiPostMarginOrder({
    #     "symbol": symbol,
    #     "side": "buy",  # "sell"
    #     "type": "market",  # limirt market
    #     "price": 1000,
    #     "quantity": 1,
    #     "sideEffectType": "MARGIN_BUY", # what?
    #     # "limit_price": None,
    #     # "stop_price": None,
    #     "timeInForce": "GTC",
    #     # "icebergQty": 0  # アイスバーグ注文？実際の注文数を隠蔽するために使う
    # })
    delete_order = await api.sapiDeleteMarginOrder({
        "orderId": 10,
        "symbol": symbol,
        # "params": {"test": True}  # ネイティブなAPIには使えない？
    })
    # {"code":-3003,"msg":"Margin account does not exist."}

    return delete_order


    # await api.create_order(
    #     symbol="BTC/USDT",
    #     type,
    #     side, amount, price=None, 
    #     params={"type": "margin"}
    # )
    # order
    # cancel
    # fetch_order_status
    # is_complete



    # ストップリミットオーダー
    # ストップ（トリガー）価格で実際に取引を行うリミット注文を発注
    # ストップとリミットを同じにすれば単なる成行のようにふるまう
    # トリガーに対して、少し下（買の場合）を指定するのが一般的？
    # めんどくさいからトリガーとリミットは一緒でよい

    # OCO
    # リミットとストップリミットの合わせ技　従来のリミットロス（利確・損切ライン）と捉える
    # 注文の仕方
    # Limit - Price - 利確価格
    # Stop-Limit - Stop - 損切り注文を発注する価格
    # Stop-Limit - Limit - 損切り価格

    # 注文プログラムの設計
    # 1. marketで注文
    # 2. OCOで利確損切り範囲を注文
    # 3. もしくは、2をキャンセルして任意のタイミングで成行決済

await main()

ExchangeError: binance {"code":-3003,"msg":"Margin account does not exist."}

In [21]:
print(1)

1
